In [28]:
import pandas as pd
import numpy as np
import re

In [8]:
df = pd.read_csv('gurgaon_properties_cleaned_v1.csv')

In [9]:
df.head()

,Property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,agePossession,nearbyLocations,furnishDetails,features
0,flat,maa bhagwati residency,sector 7,0.45,5000.0,900.0,Carpet area: 900 (83.61 sq.m.),2,2,1,not available,4.0,1 to 5 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Pear...","['3 Fan', '4 Light', '1 Wardrobe', 'No AC', 'N...","['Feng Shui / Vaastu Compliant', 'Security / F..."
1,flat,apna enclave,sector 3,0.50,7692.0,650.0,Carpet area: 650 (60.39 sq.m.),2,2,1,not available,1.0,10+ Year Old,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...","['3 Wardrobe', '4 Fan', '1 Exhaust Fan', '1 Ge...","['Security / Fire Alarm', 'Maintenance Staff',..."
2,flat,tulsiani easy in homes,sohna road,0.40,6722.0,595.0,Carpet area: 595 (55.28 sq.m.),2,2,3,not available,12.0,0 to 1 Year Old,"['Huda City Metro', 'Golf Course extn road', '...",NaN,"['Power Back-up', 'Feng Shui / Vaastu Complian..."
3,flat,smart world orchard,sector 61,1.47,12250.0,1200.0,Carpet area: 1200 (111.48 sq.m.),2,2,2,study room,2.0,Dec 2023,"['Sector 55-56 Metro station', 'Bestech Centra...",NaN,"['Security / Fire Alarm', 'Private Garden / Te..."
4,flat,parkwood westend,sector 92,0.70,5204.0,1345.0,Super Built up area 1345(124.95 sq.m.),2,2,3,study room,5.0,Under Construction,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. ...",[],NaN


In [10]:
df.shape

(3803, 16)

In [11]:
df.duplicated().sum()

124

In [12]:
df.head(1)
# focus is on -> areaWithType, additionalRoom, agePossession, furnishDetails, features

,Property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,agePossession,nearbyLocations,furnishDetails,features
0,flat,maa bhagwati residency,sector 7,0.45,5000.0,900.0,Carpet area: 900 (83.61 sq.m.),2,2,1,not available,4.0,1 to 5 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Pear...","['3 Fan', '4 Light', '1 Wardrobe', 'No AC', 'N...","['Feng Shui / Vaastu Compliant', 'Security / F..."


In [14]:
df['areaWithType']

0                          Carpet area: 900 (83.61 sq.m.)
1                          Carpet area: 650 (60.39 sq.m.)
2                          Carpet area: 595 (55.28 sq.m.)
3                        Carpet area: 1200 (111.48 sq.m.)
4                  Super Built up area 1345(124.95 sq.m.)
                              ...                        
3798    Plot area 1600(148.64 sq.m.)Built Up area: 170...
3799                          Plot area 200(167.23 sq.m.)
3800                          Plot area 150(125.42 sq.m.)
3801                          Plot area 150(125.42 sq.m.)
3802                           Plot area 109(91.14 sq.m.)
Name: areaWithType, Length: 3803, dtype: object

In [15]:
df['areaWithType'].value_counts()

Plot area 360(301.01 sq.m.)                                                                                     37
Plot area 300(250.84 sq.m.)                                                                                     26
Plot area 502(419.74 sq.m.)                                                                                     19
Plot area 200(167.23 sq.m.)                                                                                     19
Super Built up area 1350(125.42 sq.m.)                                                                          17
                                                                                                                ..
Super Built up area 1811(168.25 sq.m.)Carpet area: 1560 sq.ft. (144.93 sq.m.)                                    1
Super Built up area 1998(185.62 sq.m.)                                                                           1
Super Built up area 2600(241.55 sq.m.)Carpet area: 2100 sq.ft. (195.1 sq.m.)    

In [16]:
df['areaWithType'].isnull().sum()

0

In [25]:
df.sample(10)[['price','area','areaWithType']]

,price,area,areaWithType
2533,1.90,2835.0,Built Up area: 2835 (263.38 sq.m.)
2836,1.00,2023.0,Super Built up area 2025(188.13 sq.m.)
3423,0.70,999.0,Plot area 111(92.81 sq.m.)
1818,2.00,2338.0,Super Built up area 2338(217.21 sq.m.)
3037,6.60,1800.0,Plot area 200(167.23 sq.m.)
1134,0.71,1326.0,Carpet area: 1326 (123.19 sq.m.)
3148,8.95,5000.0,Plot area 5000(464.52 sq.m.)
576,0.75,1050.0,Super Built up area 1326(123.19 sq.m.)Built Up...
2170,1.80,1704.0,Carpet area: 1704 (158.31 sq.m.)
3286,5.50,4200.0,Plot area 4200(390.19 sq.m.)


In [54]:
def get_super_built_area(text):
    match = re.search(r'Super Built up area (\d+\.?\d*)',text)
    if match:
        return float(match.group(1))
    return None

In [56]:
# This function extracts the Built Up area or Carpet area
def get_area(text, area_type):
    match = re.search(area_type + r'\s*:\s*(\d+\.?\d*)', text)
    if match:
        return float(match.group(1))
    return None

In [57]:
# This function checks if the area is provided in sq.m. and converts it to sqft if needed
def convert_to_sqft(text, area_value):
    if area_value is None:
        return None
    match = re.search(r'{} \((\d+\.?\d*) sq.m.\)'.format(area_value), text)
    if match:
        sq_m_value = float(match.group(1))
        return sq_m_value * 10.7639  # conversion factor from sq.m. to sqft
    return area_value

In [58]:
df['super_built_up_area'] = df['areaWithType'].apply(get_super_built_area)

In [61]:
df['super_built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['super_built_up_area']), axis=1)

# Extract Built Up area and convert to sqft if needed
df['built_up_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Built Up area'))
df['built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['built_up_area']), axis=1)

# Extract Carpet area and convert to sqft if needed
df['carpet_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Carpet area'))
df['carpet_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['carpet_area']), axis=1)

In [63]:
df[['price','Property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].sample(5)

,price,Property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
433,3.50,flat,2400.0,Super Built up area 2400(222.97 sq.m.)Carpet a...,2400.0,NaN,2050.000000
3413,1.60,house,540.0,Plot area 60(50.17 sq.m.),NaN,NaN,NaN
3750,8.00,house,300.0,Plot area 300(27.87 sq.m.),NaN,NaN,NaN
2714,0.25,flat,482.0,Carpet area: 481.73 (44.75 sq.m.),NaN,NaN,481.684525
219,0.40,flat,680.0,Carpet area: 680 (63.17 sq.m.),NaN,NaN,680.000000


In [65]:
df.head(2)

,Property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,agePossession,nearbyLocations,furnishDetails,features,super_built_up_area,built_up_area,carpet_area
0,flat,maa bhagwati residency,sector 7,0.45,5000.0,900.0,Carpet area: 900 (83.61 sq.m.),2,2,1,not available,4.0,1 to 5 Year Old,"['Chintapurni Mandir', 'State bank ATM', 'Pear...","['3 Fan', '4 Light', '1 Wardrobe', 'No AC', 'N...","['Feng Shui / Vaastu Compliant', 'Security / F...",NaN,NaN,900.0
1,flat,apna enclave,sector 3,0.50,7692.0,650.0,Carpet area: 650 (60.39 sq.m.),2,2,1,not available,1.0,10+ Year Old,"['Chintapurni Mandir', 'Sheetla Mata Mandir', ...","['3 Wardrobe', '4 Fan', '1 Exhaust Fan', '1 Ge...","['Security / Fire Alarm', 'Maintenance Staff',...",NaN,NaN,650.0


In [66]:
df.shape

(3803, 19)

In [67]:
df.duplicated().sum()

124

In [69]:
df[~((df['super_built_up_area'].isnull()) | (df['built_up_area'].isnull()) | (df['carpet_area'].isnull()))][['price','Property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]

,price,Property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
38,0.78,flat,1365.0,Super Built up area 1365(126.81 sq.m.)Built Up...,1365.0,1068.1,900.0
101,2.10,flat,2000.0,Super Built up area 2000(185.81 sq.m.)Built Up...,2000.0,1600.0,1500.0
132,0.90,flat,1950.0,Super Built up area 1950(181.16 sq.m.)Built Up...,1950.0,1900.0,1854.0
135,0.90,flat,1530.0,Super Built up area 1530(142.14 sq.m.)Built Up...,1530.0,1450.0,1400.0
140,1.15,flat,2000.0,Super Built up area 2500(232.26 sq.m.)Built Up...,2500.0,2000.0,1750.0
...,...,...,...,...,...,...,...
2869,0.32,flat,687.0,Super Built up area 687(63.82 sq.m.)Built Up a...,687.0,601.0,448.0
2884,0.55,flat,950.0,Super Built up area 950(88.26 sq.m.)Built Up a...,950.0,900.0,744.0
2886,0.35,flat,867.0,Super Built up area 867(80.55 sq.m.)Built Up a...,867.0,737.0,650.0
2925,1.35,flat,1500.0,Super Built up area 1530(142.14 sq.m.)Built Up...,1530.0,1350.0,1200.0


In [82]:
df[~(df['super_built_up_area'].isnull() | df['built_up_area'].isnull() | df['carpet_area'].isnull())][['price','Property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]

,price,Property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
38,0.78,flat,1365.0,Super Built up area 1365(126.81 sq.m.)Built Up...,1365.0,1068.1,900.0
101,2.10,flat,2000.0,Super Built up area 2000(185.81 sq.m.)Built Up...,2000.0,1600.0,1500.0
132,0.90,flat,1950.0,Super Built up area 1950(181.16 sq.m.)Built Up...,1950.0,1900.0,1854.0
135,0.90,flat,1530.0,Super Built up area 1530(142.14 sq.m.)Built Up...,1530.0,1450.0,1400.0
140,1.15,flat,2000.0,Super Built up area 2500(232.26 sq.m.)Built Up...,2500.0,2000.0,1750.0
...,...,...,...,...,...,...,...
2869,0.32,flat,687.0,Super Built up area 687(63.82 sq.m.)Built Up a...,687.0,601.0,448.0
2884,0.55,flat,950.0,Super Built up area 950(88.26 sq.m.)Built Up a...,950.0,900.0,744.0
2886,0.35,flat,867.0,Super Built up area 867(80.55 sq.m.)Built Up a...,867.0,737.0,650.0
2925,1.35,flat,1500.0,Super Built up area 1530(142.14 sq.m.)Built Up...,1530.0,1350.0,1200.0


In [83]:
df[~(df['super_built_up_area'].isnull() | df['built_up_area'].isnull() | df['carpet_area'].isnull())][['price','Property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]

,price,Property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
38,0.78,flat,1365.0,Super Built up area 1365(126.81 sq.m.)Built Up...,1365.0,1068.1,900.0
101,2.10,flat,2000.0,Super Built up area 2000(185.81 sq.m.)Built Up...,2000.0,1600.0,1500.0
132,0.90,flat,1950.0,Super Built up area 1950(181.16 sq.m.)Built Up...,1950.0,1900.0,1854.0
135,0.90,flat,1530.0,Super Built up area 1530(142.14 sq.m.)Built Up...,1530.0,1450.0,1400.0
140,1.15,flat,2000.0,Super Built up area 2500(232.26 sq.m.)Built Up...,2500.0,2000.0,1750.0
...,...,...,...,...,...,...,...
2869,0.32,flat,687.0,Super Built up area 687(63.82 sq.m.)Built Up a...,687.0,601.0,448.0
2884,0.55,flat,950.0,Super Built up area 950(88.26 sq.m.)Built Up a...,950.0,900.0,744.0
2886,0.35,flat,867.0,Super Built up area 867(80.55 sq.m.)Built Up a...,867.0,737.0,650.0
2925,1.35,flat,1500.0,Super Built up area 1530(142.14 sq.m.)Built Up...,1530.0,1350.0,1200.0


In [84]:
df[~(df['super_built_up_area'].isnull() | df['built_up_area'].isnull() | df['carpet_area'].isnull())][['price','Property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]

,price,Property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
38,0.78,flat,1365.0,Super Built up area 1365(126.81 sq.m.)Built Up...,1365.0,1068.1,900.0
101,2.10,flat,2000.0,Super Built up area 2000(185.81 sq.m.)Built Up...,2000.0,1600.0,1500.0
132,0.90,flat,1950.0,Super Built up area 1950(181.16 sq.m.)Built Up...,1950.0,1900.0,1854.0
135,0.90,flat,1530.0,Super Built up area 1530(142.14 sq.m.)Built Up...,1530.0,1450.0,1400.0
140,1.15,flat,2000.0,Super Built up area 2500(232.26 sq.m.)Built Up...,2500.0,2000.0,1750.0
...,...,...,...,...,...,...,...
2869,0.32,flat,687.0,Super Built up area 687(63.82 sq.m.)Built Up a...,687.0,601.0,448.0
2884,0.55,flat,950.0,Super Built up area 950(88.26 sq.m.)Built Up a...,950.0,900.0,744.0
2886,0.35,flat,867.0,Super Built up area 867(80.55 sq.m.)Built Up a...,867.0,737.0,650.0
2925,1.35,flat,1500.0,Super Built up area 1530(142.14 sq.m.)Built Up...,1530.0,1350.0,1200.0


In [88]:
df[(df['super_built_up_area'].isnull() & df['built_up_area'].isnull() & df['carpet_area'].isnull())][['price','Property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]

,price,Property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2943,5.25,house,2610.0,Plot area 290(242.48 sq.m.),NaN,NaN,NaN
2944,2.10,house,549.0,Plot area 61(51 sq.m.),NaN,NaN,NaN
2945,5.00,house,1161.0,Plot area 129(107.86 sq.m.),NaN,NaN,NaN
2946,3.00,house,558.0,Plot area 62(51.84 sq.m.),NaN,NaN,NaN
2947,4.50,house,1152.0,Plot area 128(107.02 sq.m.),NaN,NaN,NaN
...,...,...,...,...,...,...,...
3797,3.10,house,1548.0,Plot area 172(143.81 sq.m.),NaN,NaN,NaN
3799,5.50,house,1800.0,Plot area 200(167.23 sq.m.),NaN,NaN,NaN
3800,4.25,house,1350.0,Plot area 150(125.42 sq.m.),NaN,NaN,NaN
3801,4.50,house,1350.0,Plot area 150(125.42 sq.m.),NaN,NaN,NaN


In [94]:
df[df['areaWithType'].str.contains('Plot')][['price','Property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].head(5)

,price,Property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2943,5.25,house,2610.0,Plot area 290(242.48 sq.m.),NaN,NaN,NaN
2944,2.10,house,549.0,Plot area 61(51 sq.m.),NaN,NaN,NaN
2945,5.00,house,1161.0,Plot area 129(107.86 sq.m.),NaN,NaN,NaN
2946,3.00,house,558.0,Plot area 62(51.84 sq.m.),NaN,NaN,NaN
2947,4.50,house,1152.0,Plot area 128(107.02 sq.m.),NaN,NaN,NaN


In [95]:
df.isnull().sum()

Property_type             0
society                   2
sector                    0
price                    18
price_per_sqft           18
area                     18
areaWithType              0
bedRoom                   0
bathroom                  0
balcony                   0
additionalRoom            0
floorNum                 19
agePossession             1
nearbyLocations         177
furnishDetails          981
features                635
super_built_up_area    1888
built_up_area          2616
carpet_area            1859
dtype: int64

In [104]:
all_nan_df = df[((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]
all_nan_df

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2943,5.25,house,2610.0,Plot area 290(242.48 sq.m.),NaN,NaN,NaN
2944,2.10,house,549.0,Plot area 61(51 sq.m.),NaN,NaN,NaN
2945,5.00,house,1161.0,Plot area 129(107.86 sq.m.),NaN,NaN,NaN
2946,3.00,house,558.0,Plot area 62(51.84 sq.m.),NaN,NaN,NaN
2947,4.50,house,1152.0,Plot area 128(107.02 sq.m.),NaN,NaN,NaN
...,...,...,...,...,...,...,...
3797,3.10,house,1548.0,Plot area 172(143.81 sq.m.),NaN,NaN,NaN
3799,5.50,house,1800.0,Plot area 200(167.23 sq.m.),NaN,NaN,NaN
3800,4.25,house,1350.0,Plot area 150(125.42 sq.m.),NaN,NaN,NaN
3801,4.50,house,1350.0,Plot area 150(125.42 sq.m.),NaN,NaN,NaN


In [106]:
all_nan_df.head()

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2943,5.25,house,2610.0,Plot area 290(242.48 sq.m.),NaN,NaN,NaN
2944,2.10,house,549.0,Plot area 61(51 sq.m.),NaN,NaN,NaN
2945,5.00,house,1161.0,Plot area 129(107.86 sq.m.),NaN,NaN,NaN
2946,3.00,house,558.0,Plot area 62(51.84 sq.m.),NaN,NaN,NaN
2947,4.50,house,1152.0,Plot area 128(107.02 sq.m.),NaN,NaN,NaN


In [108]:
df[((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].index

Int64Index([2943, 2944, 2945, 2946, 2947, 2948, 2949, 2950, 2951, 2952,
            ...
            3792, 3793, 3794, 3795, 3796, 3797, 3799, 3800, 3801, 3802],
           dtype='int64', length=546)

In [ ]:
....
.....
....
..
..
.
.
.
.
.
.
.